In [1]:
from random import randint
import pandas as pd
import numpy as np
import scipy.special as sp
import math 
import time
from tqdm.notebook import tqdm_notebook
pd.set_option('display.max_columns', 50)

In [2]:
def PlayGame(NrOfPlayers = 2, NrOfPlayerCards = 2, NrOfSuits = 4, NrOfValues = 13, NrOfFlopCards = 3, NrOfTurnCards = 1, NrOfRiverCards = 1):
    
    # Check player and card parameter
    if (NrOfPlayers<2)|(NrOfPlayers>10):
        print('Please enter valid number of players')
        return
    if (NrOfSuits<1):
        print('Please enter valid number of suits')
        return
    if (NrOfValues<5):
        print('Please enter valid number of values')
        return
    if (NrOfPlayerCards<1)|(NrOfPlayerCards>((NrOfSuits*NrOfValues)/NrOfPlayers)):
        print('Please enter valid number of player cards')
        return
    
    # Log parameters to use in other calls
    ParamsDict = {'NrOfPlayers'  : NrOfPlayers, 'NrOfPlayerCards' : NrOfPlayerCards, 'NrOfSuits' : NrOfSuits, 'NrOfValues' : NrOfValues,
                  'NrOfFlopCards' : NrOfFlopCards, 'NrOfTurnCards' : NrOfTurnCards, 'NrOfRiverCards' : NrOfRiverCards}
    
    # Draw Player and Table cards
    PlayerHands, TableCards, ParamsDict = DrawCards(ParamsDict)
    print(PlayerHands)
    print(TableCards)
    
    # For every stage in game
    for stage in TableCards:
        
        #Combine TableCards from all stages up until current stage in one list
        TableList = list()
        Flag = True
        for stage2 in TableCards:
            if Flag:
                for card in TableCards[stage2]:
                    TableList.append(card)
                if stage2 == stage:
                    Flag = False
        
        #Combine Player Cards and Table Cards in dict one List per player 
        PlayerAvailCards = dict()
        for currplayer in PlayerHands:
            CurrPlayerAvailCards = list()
            for card in PlayerHands[currplayer]:
                CurrPlayerAvailCards.append(card)
            for card in TableList:
                CurrPlayerAvailCards.append(card)
            PlayerAvailCards[currplayer] = CurrPlayerAvailCards
        
        CheckPlayerProbabilities(PlayerAvailCards, ParamsDict, stage)
        # CheckTableProbabilities(TableList, ParamsDict, stage)
        
            
    
    return ParamsDict

In [3]:
def DrawCards( ParamsDict ):
    
    NrOfValues = ParamsDict['NrOfValues']
    NrOfSuits = ParamsDict['NrOfSuits']
    NrOfPlayers = ParamsDict['NrOfPlayers']
    NrOfPlayerCards = ParamsDict['NrOfPlayerCards']
    NrOfTableCards = {'Flop' : ParamsDict['NrOfFlopCards'], 'Turn' : ParamsDict['NrOfTurnCards'], 'River' : ParamsDict['NrOfRiverCards']}
    
    # Set up card deck
    Suits = ['A','B','C','D','E','F','G','H']
    Suits = [Suits[i] for i in list(range(0,NrOfSuits))]
    Values = list(range(2, NrOfValues+2))
    Deck = [(suit, value) for suit in Suits for value in Values]
    AvailableCards = list(range(0,len(Deck)))
    
    # Draw Player cards
    PlayerHands = dict()
    for i in range(1,NrOfPlayers+1):
        CurrPlayerCards = list()
        for j in range(0,NrOfPlayerCards):
            RandIndex = randint(0,len(AvailableCards)-1)
            RandIndexInCardPool = AvailableCards[RandIndex]
            AvailableCards.remove(RandIndexInCardPool)
            DrawnCard = Deck[RandIndexInCardPool]
            CurrPlayerCards.append(DrawnCard)
        PlayerHands['Player_'+str(i)] = CurrPlayerCards           
    
    # Draw Table cards
    TableCards = {'Flop' : list(), 'Turn': list(), 'River': list()}
    for stage in TableCards:
        StageCards = list()
        for i in range(0,NrOfTableCards[stage]):
            RandIndex = randint(0,len(AvailableCards)-1)
            RandIndexInCardPool = AvailableCards[RandIndex]
            AvailableCards.remove(RandIndexInCardPool)
            DrawnCard = Deck[RandIndexInCardPool]
            StageCards.append(DrawnCard)
        TableCards[stage] = StageCards 
    
    # Add Params to ParamsDict for other functions
    ParamsDict['Suits'] = Suits
    ParamsDict['Values'] = Values
    ParamsDict['Deck'] = Deck
    
    return PlayerHands, TableCards, ParamsDict

In [4]:
def CheckPlayerProbabilities( PlayerAvailCards, ParamsDict, Stage ):
    
    NrOfTableCards = {'Flop' : ParamsDict['NrOfFlopCards'], 'Turn' : ParamsDict['NrOfTurnCards'], 'River' : ParamsDict['NrOfRiverCards']}

    # Count Nr of Cards left
    NrOfCardsLeft = 0
    Flag = False
    for stage in NrOfTableCards:
        if Flag:
            NrOfCardsLeft += NrOfTableCards[stage]
        if stage == Stage:
            Flag = True
            
    for currplayer in PlayerAvailCards:    
        CheckProbabilityFlush( PlayerAvailCards[currplayer], ParamsDict, NrOfCardsLeft)
    
    return

In [29]:
def CheckProbabilityFlush( AvailCardsList, ParamsDict, NrOfCardsLeft):
    
    Suits = ParamsDict['Suits']
    NrOfSuits = ParamsDict['NrOfSuits']
    NrOfValues = ParamsDict['NrOfValues']
    NrOfCardsInDeck = NrOfSuits * NrOfValues
    NrOfPlayers = ParamsDict['NrOfPlayers']
    NrOfPlayerCards = ParamsDict['NrOfPlayerCards']
    
    SuitsList = [card[0] for card in AvailCardsList]
    SuitsArr = np.zeros(4,'int')
    for (index, suit) in enumerate(Suits):
        SuitsArr[index] = SuitsList.count(suit)
    FlushArr = np.zeros(4,'int')
    FlushArr.fill(5)
    FlushArr = np.subtract(FlushArr,SuitsArr)
    FlushInd = -1
    for (index, cardsleft) in enumerate(FlushArr):
        if cardsleft>NrOfCardsLeft:
            FlushArr[index] = 0
        if cardsleft<=NrOfCardsLeft:
            FlushInd = index
            
    if FlushInd != -1:
        
        
        #to rework
        # Calculate probability that other players have drawn cards from suit
        InSuitCardsLeft = NrOfValues - SuitsArr[FlushInd]
        OffSuitCardsLeft = NrOfCardsInDeck - NrOfValues - (len(AvailCardsList)-SuitsArr[FlushInd])
        TotCardsLeft = InSuitCardsLeft + OffSuitCardsLeft
    
        ProbCardsDrawnArr = np.zeros(InSuitCardsLeft+1,'float')
        for (index, value) in enumerate(ProbCardsDrawnArr):
            if index<=2*(NrOfPlayers-1):
                ProbCardsDrawnArr[index] = sp.comb(2*(NrOfPlayers-1),index,exact=True)*(math.factorial(InSuitCardsLeft)/math.factorial(InSuitCardsLeft-index))*(math.factorial(OffSuitCardsLeft)/math.factorial(OffSuitCardsLeft-(2*(NrOfPlayers-1)-index)))*(math.factorial(TotCardsLeft-2*(NrOfPlayers-1))/math.factorial(TotCardsLeft))
            
        print(InSuitCardsLeft)
        print(OffSuitCardsLeft)
        print(TotCardsLeft)
        print(InSuitCardsLeft/TotCardsLeft)
        print(ProbCardsDrawnArr)
        print(np.sum(ProbCardsDrawnArr))
        ProbCardsDrawnArr *= np.arange(0,InSuitCardsLeft+1)
        print(ProbCardsDrawnArr)
        print(np.sum(ProbCardsDrawnArr))
        
        AvgInSuitCardsLeft = InSuitCardsLeft - np.sum(ProbCardsDrawnArr)
        TotTableCardsLeft = TotCardsLeft - (2*(NrOfPlayers-1))
        Ratio = AvgInSuitCardsLeft/TotTableCardsLeft
        
        print(AvgInSuitCardsLeft)
        print(TotTableCardsLeft)
        print(Ratio)
        
    return 

In [23]:
prd = PlayGame(NrOfPlayers = 2)

{'Player_1': [('A', 13), ('A', 9)], 'Player_2': [('B', 14), ('D', 13)]}
{'Flop': [('D', 12), ('D', 3), ('D', 6)], 'Turn': [('A', 5)], 'River': [('B', 6)]}
10
37
[0.61609621 0.34227567 0.04162812 0.         0.         0.
 0.         0.         0.         0.         0.        ]
1.0
[0.         0.34227567 0.08325624 0.         0.         0.
 0.         0.         0.         0.         0.        ]
0.4255319148936171
0.21276595744680854
9
38
[0.65032377 0.31637373 0.0333025  0.         0.         0.
 0.         0.         0.         0.        ]
1.0000000000000002
[0.         0.31637373 0.066605   0.         0.         0.
 0.         0.         0.         0.        ]
0.3829787234042553
0.19148936170212766
9
37
[0.64347826 0.32173913 0.03478261 0.         0.         0.
 0.         0.         0.         0.        ]
1.0
[0.         0.32173913 0.06956522 0.         0.         0.
 0.         0.         0.         0.        ]
0.3913043478260869
0.1956521739130435


In [30]:
acl = [('D', 14), ('B', 7), ('A', 12), ('B', 13), ('B', 9)]
CheckProbabilityFlush( acl, prd, 2)

10
37
47
0.2127659574468085
[0.61609621 0.34227567 0.04162812 0.         0.         0.
 0.         0.         0.         0.         0.        ]
1.0
[0.         0.34227567 0.08325624 0.         0.         0.
 0.         0.         0.         0.         0.        ]
0.4255319148936171
9.574468085106384
45
0.21276595744680854


In [12]:
np.arange(0,7)

array([0, 1, 2, 3, 4, 5, 6])